# 🌍 Environmental Policy Impact Agent System

**Kaggle AI Agents Intensive Capstone Project**

**Track A: Consent Agents** | **Team Robee**

---

This notebook demonstrates our multi-agent system for analyzing environmental policy effectiveness.

## Core Concepts Applied
- ✅ **Day 1**: Multi-Agent System Architecture
- ✅ **Day 2**: Custom Tools & MCP Integration
- ✅ **Day 3**: Memory & Context Engineering

## 1. Setup & Installation

In [ ]:
# Install dependencies
!pip install -q httpx python-dotenv

In [ ]:
# Import libraries
import os
import json
import statistics
from datetime import datetime
from typing import Dict, Any, List, Optional

print("✅ Libraries imported successfully!")

## 2. Custom Tools (Day 2 Concept)

We define custom tools that our agents will use.

In [ ]:
# Tool 1: AQI Helper Functions

def get_aqi_category(aqi: int) -> str:
    """Get AQI category label."""
    if aqi <= 50: return "Good"
    elif aqi <= 100: return "Moderate"
    elif aqi <= 150: return "Unhealthy for Sensitive Groups"
    elif aqi <= 200: return "Unhealthy"
    elif aqi <= 300: return "Very Unhealthy"
    else: return "Hazardous"

def get_aqi_color(aqi: int) -> str:
    """Get AQI color code."""
    if aqi <= 50: return "#00e400"
    elif aqi <= 100: return "#ffff00"
    elif aqi <= 150: return "#ff7e00"
    elif aqi <= 200: return "#ff0000"
    elif aqi <= 300: return "#8f3f97"
    else: return "#7e0023"

print("✅ AQI helper functions defined!")

In [ ]:
# Tool 2: Air Quality Data Tool (Demo)

def fetch_air_quality_demo(location: str) -> Dict[str, Any]:
    """
    Fetch air quality data (demo version).
    In production, this calls WAQI API.
    """
    # Demo data for different countries
    demo_data = {
        "South Korea": {"aqi": 45, "pm25": 28, "pm10": 45},
        "China": {"aqi": 125, "pm25": 78, "pm10": 120},
        "Japan": {"aqi": 35, "pm25": 18, "pm10": 30},
        "India": {"aqi": 185, "pm25": 135, "pm10": 180},
    }
    
    data = demo_data.get(location, {"aqi": 50, "pm25": 30, "pm10": 50})
    aqi = data["aqi"]
    
    return {
        "location": location,
        "aqi": aqi,
        "pm25": data["pm25"],
        "pm10": data["pm10"],
        "category": get_aqi_category(aqi),
        "color": get_aqi_color(aqi),
        "timestamp": datetime.now().isoformat(),
        "status": "success"
    }

# Test
result = fetch_air_quality_demo("South Korea")
print(f"🇰🇷 South Korea AQI: {result['aqi']} ({result['category']})")

In [ ]:
# Tool 3: Policy Database Tool

POLICIES_DB = [
    {
        "id": "kr_2023_emission",
        "name": "2023 Emission Reduction Act",
        "country": "South Korea",
        "enacted_date": "2023-06-01",
        "target_reduction": "30%",
        "status": "active"
    },
    {
        "id": "kr_2019_fine_dust",
        "name": "Comprehensive Fine Dust Management Act",
        "country": "South Korea",
        "enacted_date": "2019-02-15",
        "target_reduction": "35%",
        "actual_reduction": "37%",
        "status": "completed"
    },
    {
        "id": "cn_2020_blue_sky",
        "name": "Beijing Blue Sky Defense Action Plan",
        "country": "China",
        "enacted_date": "2020-01-01",
        "target_reduction": "25%",
        "actual_reduction": "23%",
        "status": "completed"
    }
]

def search_policies(country: str) -> Dict[str, Any]:
    """Search environmental policies for a country."""
    policies = [p for p in POLICIES_DB if p["country"] == country]
    return {
        "country": country,
        "policies": policies,
        "count": len(policies),
        "status": "success"
    }

# Test
result = search_policies("South Korea")
print(f"Found {result['count']} policies for South Korea")
for p in result['policies']:
    print(f"  • {p['name']} ({p['status']})")

In [ ]:
# Tool 4: Statistical Analysis Tool

def compare_before_after(
    before_data: List[float],
    after_data: List[float],
    metric_name: str = "PM2.5"
) -> Dict[str, Any]:
    """Compare data before and after policy implementation."""
    
    before_mean = statistics.mean(before_data)
    after_mean = statistics.mean(after_data)
    
    absolute_change = after_mean - before_mean
    percent_change = (absolute_change / before_mean * 100) if before_mean != 0 else 0
    
    # Effect size (Cohen's d)
    before_std = statistics.stdev(before_data) if len(before_data) > 1 else 0
    after_std = statistics.stdev(after_data) if len(after_data) > 1 else 0
    pooled_std = ((before_std**2 + after_std**2) / 2) ** 0.5
    cohens_d = abs(before_mean - after_mean) / pooled_std if pooled_std > 0 else 0
    
    if cohens_d < 0.2: effect = "negligible"
    elif cohens_d < 0.5: effect = "small"
    elif cohens_d < 0.8: effect = "medium"
    else: effect = "large"
    
    return {
        "metric": metric_name,
        "before_mean": round(before_mean, 2),
        "after_mean": round(after_mean, 2),
        "percent_change": round(percent_change, 2),
        "improvement": after_mean < before_mean,
        "cohens_d": round(cohens_d, 3),
        "effect_size": effect,
        "status": "success"
    }

# Test with sample data
before = [38, 40, 37, 39, 38.5, 41, 36, 39]
after = [24, 25, 23, 24.5, 24, 26, 22, 25]

result = compare_before_after(before, after)
print(f"📊 Analysis Results:")
print(f"   Before: {result['before_mean']} μg/m³")
print(f"   After: {result['after_mean']} μg/m³")
print(f"   Change: {result['percent_change']}%")
print(f"   Effect Size: {result['effect_size']}")

## 3. Memory System (Day 3 Concept)

Session and long-term memory for context engineering.

In [ ]:
# Session Memory

class SessionManager:
    """Manages session-level memory."""
    
    def __init__(self):
        self._preferences = {}
        self._history = []
        self._context = {}
    
    def set_preference(self, key: str, value: Any):
        self._preferences[key] = value
    
    def get_preference(self, key: str, default=None):
        return self._preferences.get(key, default)
    
    def add_to_history(self, query: str, result: Any):
        self._history.append({
            "query": query,
            "result": result,
            "timestamp": datetime.now().isoformat()
        })
    
    def get_history(self, limit: int = 5):
        return self._history[-limit:]

# Test session memory
session = SessionManager()
session.set_preference("favorite_countries", ["South Korea", "Japan"])
session.set_preference("language", "ko")

print(f"✅ Session initialized")
print(f"   Favorite countries: {session.get_preference('favorite_countries')}")

In [ ]:
# Long-term Memory (simplified)

class LongTermMemory:
    """Manages persistent long-term memory."""
    
    def __init__(self):
        self._analyses = []
        self._stats = {"total": 0}
    
    def save_analysis(self, result: Dict):
        self._analyses.append({
            "result": result,
            "saved_at": datetime.now().isoformat()
        })
        self._stats["total"] += 1
    
    def get_past_analyses(self, country: str = None, limit: int = 5):
        analyses = self._analyses
        if country:
            analyses = [
                a for a in analyses 
                if a["result"].get("country") == country
            ]
        return analyses[-limit:]
    
    def get_stats(self):
        return self._stats

# Initialize memory
memory = LongTermMemory()
print("✅ Long-term memory initialized")

## 4. Multi-Agent System (Day 1 Concept)

Four specialized agents working together.

In [ ]:
# Agent Definitions

class DataCollectorAgent:
    """Collects air quality and policy data."""
    name = "DataCollector"
    
    def collect(self, country: str):
        print(f"📡 [{self.name}] Collecting data for {country}...")
        air_quality = fetch_air_quality_demo(country)
        policies = search_policies(country)
        return {"air_quality": air_quality, "policies": policies}


class PolicyAnalyzerAgent:
    """Analyzes policy effectiveness."""
    name = "PolicyAnalyzer"
    
    def analyze(self, before: List[float], after: List[float]):
        print(f"📊 [{self.name}] Analyzing policy impact...")
        return compare_before_after(before, after)


class VisualizerAgent:
    """Creates visualizations."""
    name = "Visualizer"
    
    def create_summary_card(self, data: Dict):
        print(f"🎨 [{self.name}] Creating visualization...")
        return {
            "type": "summary_card",
            "before": data.get("before_mean"),
            "after": data.get("after_mean"),
            "change": data.get("percent_change")
        }


class ReporterAgent:
    """Generates reports."""
    name = "Reporter"
    
    def generate_summary(self, country: str, analysis: Dict):
        print(f"📄 [{self.name}] Generating report...")
        
        change = analysis.get("percent_change", 0)
        improvement = analysis.get("improvement", False)
        
        lines = [
            f"📊 {country} 환경 정책 분석 결과",
            "",
            "【주요 결과】",
            f"• PM2.5: {analysis.get('before_mean')} → {analysis.get('after_mean')} μg/m³",
        ]
        
        if improvement:
            lines.append(f"• 변화율: ▼ {abs(change):.1f}% 감소 (개선)")
        else:
            lines.append(f"• 변화율: ▲ {abs(change):.1f}% 증가")
        
        lines.append(f"• 효과 크기: {analysis.get('effect_size')}")
        
        return "\n".join(lines)

print("✅ Agents defined!")

In [ ]:
# Orchestrator: Coordinates all agents

class PolicyAgentSystem:
    """Main orchestrator for the multi-agent system."""
    
    def __init__(self):
        self.data_collector = DataCollectorAgent()
        self.analyzer = PolicyAnalyzerAgent()
        self.visualizer = VisualizerAgent()
        self.reporter = ReporterAgent()
        self.session = SessionManager()
        self.memory = LongTermMemory()
    
    def analyze_policy(self, country: str, policy_name: str = None):
        """Run full policy analysis."""
        print(f"\n{'='*60}")
        print(f"🌍 Analyzing {country}'s environmental policy")
        print(f"{'='*60}\n")
        
        # Step 1: Data Collection
        data = self.data_collector.collect(country)
        
        # Step 2: Analysis (using demo data)
        before = [38, 40, 37, 39, 38.5, 41, 36, 39]
        after = [24, 25, 23, 24.5, 24, 26, 22, 25]
        analysis = self.analyzer.analyze(before, after)
        
        # Step 3: Visualization
        viz = self.visualizer.create_summary_card(analysis)
        
        # Step 4: Report
        report = self.reporter.generate_summary(country, analysis)
        
        # Compile result
        result = {
            "country": country,
            "policy": policy_name or "Environmental Policy",
            "current_air_quality": data["air_quality"],
            "policies_found": data["policies"],
            "analysis": analysis,
            "visualization": viz,
            "report": report
        }
        
        # Save to memory (Day 3)
        self.memory.save_analysis(result)
        self.session.add_to_history(f"Analyze {country}", result)
        
        return result

# Initialize system
system = PolicyAgentSystem()
print("✅ Multi-Agent System ready!")

## 5. Demo: Analyze South Korea's Policy

In [ ]:
# Run analysis
result = system.analyze_policy(
    country="South Korea",
    policy_name="2023 Emission Reduction Act"
)

In [ ]:
# Display report
print("\n" + "="*60)
print("📋 EXECUTIVE SUMMARY")
print("="*60)
print(result["report"])

In [ ]:
# Show current air quality
aq = result["current_air_quality"]
print(f"\n🌡️ Current Air Quality:")
print(f"   AQI: {aq['aqi']} ({aq['category']})")
print(f"   PM2.5: {aq['pm25']} μg/m³")
print(f"   PM10: {aq['pm10']} μg/m³")

In [ ]:
# Show analysis details
analysis = result["analysis"]
print(f"\n📊 Statistical Analysis:")
print(f"   Before Policy: {analysis['before_mean']} μg/m³")
print(f"   After Policy: {analysis['after_mean']} μg/m³")
print(f"   Change: {analysis['percent_change']}%")
print(f"   Improvement: {'✅ Yes' if analysis['improvement'] else '❌ No'}")
print(f"   Effect Size: {analysis['effect_size']} (Cohen's d = {analysis['cohens_d']})")

## 6. Demo: Multi-Country Comparison

In [ ]:
# Compare multiple countries
countries = ["South Korea", "China", "Japan", "India"]

print("🌍 Multi-Country Air Quality Comparison")
print("="*50)

comparison_data = []
for country in countries:
    data = fetch_air_quality_demo(country)
    comparison_data.append(data)
    print(f"{country}: AQI {data['aqi']} ({data['category']})")

# Sort by AQI (lower is better)
comparison_data.sort(key=lambda x: x['aqi'])

print("\n🏆 Ranking (Best to Worst):")
for i, data in enumerate(comparison_data, 1):
    print(f"   {i}. {data['location']}: {data['aqi']}")

## 7. Memory Demo (Day 3)

In [ ]:
# Check memory
print("📚 Memory Status:")
print(f"   Total analyses saved: {system.memory.get_stats()['total']}")
print(f"   Session history: {len(system.session.get_history())} queries")

# Show history
print("\n📜 Query History:")
for item in system.session.get_history():
    print(f"   • {item['query']} ({item['timestamp'][:19]})")

## 📝 Summary

This notebook demonstrated:

### Day 1: Multi-Agent System
- 4 specialized agents (DataCollector, PolicyAnalyzer, Visualizer, Reporter)
- Orchestrator coordinating all agents

### Day 2: Custom Tools
- Air Quality Tool (WAQI API demo)
- Policy Database Tool
- Statistical Analysis Tool

### Day 3: Memory & Context
- Session Memory for query history
- Long-term Memory for persistent storage

---

**GitHub**: [joymin5655/Kaggle](https://github.com/joymin5655/Kaggle)

**Team Robee** 🐝